In [1]:
from agent_framework.tools.mcp_tool import MCPTool, MCPClient
from rich.pretty import pprint

client = MCPClient()
await client.connect_sse('http://localhost:8000/sse')
mcp_tools = await MCPTool.from_mcp_client(client)

{"timestamp": "2026-02-07T12:58:09.349767", "level": "INFO", "name": "httpx", "message": "HTTP Request: POST http://localhost:8000/messages/?session_id=a2f9ba41404f4a11ab379352bd3578f7 \"HTTP/1.1 202 Accepted\""}
{"timestamp": "2026-02-07T12:58:12.705422", "level": "INFO", "name": "httpx", "message": "HTTP Request: POST http://localhost:8000/messages/?session_id=a2f9ba41404f4a11ab379352bd3578f7 \"HTTP/1.1 202 Accepted\""}


In [2]:
from agent_framework.model_clients.openai.openai_client import OpenAIClient
from agent_framework.messages.client_messages import UserMessage
from agent_framework.configs.settings import settings
from agent_framework.observability.telemetry import configure_opentelemetry

# Fix: Use HTTP endpoint for traces (4318) and metrics  
configure_opentelemetry(
    service_name="agent-framework-client-test", 
    otlp_trace_endpoint="http://localhost:4318",  # Changed from 4317 (gRPC) to 4318 (HTTP)
    otlp_metric_endpoint="http://localhost:4318"
)

model_client = OpenAIClient(model="gpt-4o-mini", api_key=settings.OPENAI_API_KEY)
# response = await model_client.generate([UserMessage(content=["What is the capital of France?"])], tools=mcp_tools)
# pprint(response)

{"timestamp": "2026-02-07T12:58:07.826917", "level": "INFO", "name": "agent_framework", "message": "Configuring OpenTelemetry"}
{"timestamp": "2026-02-07T12:58:07.828569", "level": "INFO", "name": "agent_framework", "message": "Using OTLP HTTP trace exporter → http://localhost:4318/v1/traces"}


In [3]:
# response = await model_client.generate([UserMessage(content=["Can you search for Ravikumarchavva github profile and find top 3 repositories?"])], tools=mcp_tools)
# pprint(response)

In [4]:
from agent_framework.agents.react_agent import ReActAgent

agent = ReActAgent(
    name="Test ReAct Agent",
    description="A test agent for ReAct framework",
    model_client=model_client, 
    tools=mcp_tools,
    system_instructions="Answer the user's question using the available tools. If you need to use a tool, call it with the appropriate arguments. If you have the answer, respond directly to the user.",
)

In [5]:
result = await agent.run("Can you search for Ravikumarchavva github profile and find top 3 repositories?")
print(result)          # clean __str__
print()
print(result.summary())  # one-liner

{"timestamp": "2026-02-07T12:58:07.890986", "level": "INFO", "name": "agent_framework", "message": "[Test ReAct Agent] Starting run: Can you search for Ravikumarchavva github profile and find top 3 repositories?..."}
{"timestamp": "2026-02-07T12:58:09.252862", "level": "INFO", "name": "httpx", "message": "HTTP Request: POST https://api.openai.com/v1/responses \"HTTP/1.1 200 OK\""}
{"timestamp": "2026-02-07T12:58:09.344050", "level": "INFO", "name": "agent_framework", "message": "[Test ReAct Agent] Step 1: tool calls → ['web_search_tool']"}
{"timestamp": "2026-02-07T12:58:09.345158", "level": "INFO", "name": "agent_framework", "message": "[Test ReAct Agent] Executing web_search_tool({'url': 'https://github.com/Ravikumarchavva', 'user_agent': 'Mozilla/5.0', 'max_chars': 1000, 'start_index': 0})"}
{"timestamp": "2026-02-07T12:58:12.697065", "level": "INFO", "name": "httpx", "message": "HTTP Request: POST https://api.openai.com/v1/responses \"HTTP/1.1 200 OK\""}
{"timestamp": "2026-02-07T1

{"timestamp": "2026-02-07T12:59:07.841185", "level": "WARNING", "name": "opentelemetry.exporter.otlp.proto.grpc.exporter", "message": "Transient error StatusCode.UNAVAILABLE encountered while exporting metrics to localhost:4318, retrying in 0.83s."}
{"timestamp": "2026-02-07T12:59:15.659263", "level": "ERROR", "name": "opentelemetry.exporter.otlp.proto.grpc.exporter", "message": "Failed to export metrics to localhost:4318, error code: StatusCode.UNAVAILABLE"}
{"timestamp": "2026-02-07T13:00:15.661472", "level": "WARNING", "name": "opentelemetry.exporter.otlp.proto.grpc.exporter", "message": "Transient error StatusCode.UNAVAILABLE encountered while exporting metrics to localhost:4318, retrying in 1.15s."}
{"timestamp": "2026-02-07T13:00:23.329267", "level": "ERROR", "name": "opentelemetry.exporter.otlp.proto.grpc.exporter", "message": "Failed to export metrics to localhost:4318, error code: StatusCode.UNAVAILABLE"}
{"timestamp": "2026-02-07T13:01:23.331141", "level": "WARNING", "name": 

In [7]:
pprint(result)

AgentRunResult(
│   run_id='ae02dca8-f6d1-4703-9259-67ee07769c21',
│   agent_name='Test ReAct Agent',
│   output=[
│   │   "Here are the top 3 repositories from Ravikumar Chavva's GitHub profile:\n\n1. **[agentic_ai](https://github.com/Ravikumarchavva/agentic_ai)**  \n   - Description: Custom chatbots  \n   - Language: Python  \n   - Last Updated: January 19, 2026\n\n2. **[csrio-image2biomass](https://github.com/Ravikumarchavva/csrio-image2biomass)**  \n   - Description: Predicting the biomass content useful for grazing  \n   - Language: Jupyter Notebook  \n   - Last Updated: November 1, 2025\n\n3. **[computer-science](https://github.com/Ravikumarchavva/computer-science)**  \n   - Description: From 0,1 to distributed computing  \n   - Language: Jupyter Notebook  \n   - Last Updated: September 14, 2025\n\nFeel free to explore these repositories for more information!"
│   ],
│   status=<RunStatus.COMPLETED: 'completed'>,
│   steps=[
│   │   StepResult(
│   │   │   step=1,
│   │   │   thought=None,
│   │   │   tool_calls=[
│   │   │   │   ToolCallRecord(
│   │   │   │   │   tool_name='web_search_tool',
│   │   │   │   │   call_id='call_vlT4xhkGlO2xMXUToPVb1xR4',
│   │   │   │   │   arguments={
│   │   │   │   │   │   'url': 'https://github.com/Ravikumarchavva',
│   │   │   │   │   │   'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
│   │   │   │   │   │   'max_chars': 1000,
│   │   │   │   │   │   'start_index': 0
│   │   │   │   │   },
│   │   │   │   │   result="[![View Ravikumarchavva's full-sized avatar](https://avatars.githubusercontent.com/u/109415623?v=4)](https://avatars.githubusercontent.com/u/109415623?v=4)\n\n## C.Ravi Kumar reddy Ravikumarchavva\n\n## Block or report Ravikumarchavva\n\n## Hi there! 👋\n\nI’m **Ravikumar Chavva**, a final-year Computer Science and Engineering student specializing in **Data Science** and **Machine Learning**. I’m passionate about leveraging data and AI to solve real-world problems and creating impactful solutions.\n\n---\n\n## 👩\u200d💻 About Me\n\n* 🎓 **Final-Year Student**: Pursuing B.Tech with a focus on Data Science and Machine Learning.\n* 🤖 **AI Enthusiast**: Passionate about exploring cutting-edge technologies and their practical applications.\n* 🌟 **Driven Problem-Solver**: Dedicated to building end-to-end solutions integrating ML, cloud, and full-stack technologies.\n\n---\n\n## 🛠️ Skills\n\n### Programming Languages\n\n* **Python** | **SQL** | **TypeScript**\n\n### Frameworks & Tools\n\n* **Machine Learning & AI:** Scikit-lear",
│   │   │   │   │   is_error=False,
│   │   │   │   │   duration_ms=2096.7918620008277,
│   │   │   │   │   timestamp=datetime.datetime(2026, 2, 7, 12, 34, 35, 859476)
│   │   │   │   )
│   │   │   ],
│   │   │   usage=UsageStats(prompt_tokens=152, completion_tokens=87, total_tokens=239),
│   │   │   finish_reason='tool_calls',
│   │   │   has_tool_calls=True,
│   │   │   thought_text=None
│   │   ),
│   │   StepResult(
│   │   │   step=2,
│   │   │   thought=None,
│   │   │   tool_calls=[
│   │   │   │   ToolCallRecord(
│   │   │   │   │   tool_name='web_search_tool',
│   │   │   │   │   call_id='call_OlmeX5HXYQ58QmqJ242RpjOY',
│   │   │   │   │   arguments={
│   │   │   │   │   │   'url': 'https://github.com/Ravikumarchavva?tab=repositories',
│   │   │   │   │   │   'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
│   │   │   │   │   │   'max_chars': 1000,
│   │   │   │   │   │   'start_index': 0
│   │   │   │   │   },
│   │   │   │   │   result="[![View Ravikumarchavva's full-sized avatar](https://avatars.githubusercontent.com/u/109415623?v=4)](https://avatars.githubusercontent.com/u/109415623?v=4)\n\n## C.Ravi Kumar reddy Ravikumarchavva\n\n## Block or report Ravikumarchavva\n\n* ### [agentic\\_ai](/Ravikumarchavva/agentic_ai) Public\n\n  custom chatbots\n\n  Python\n  Updated\n\n  Jan 

In [9]:
print(result.summary())

[completed] Test ReAct Agent | 3/10 steps | 2 tool calls (web_search_toolx2) | 1942 tokens | 11.42s


In [12]:
pprint(agent.memory.get_messages())

[
│   SystemMessage(
│   │   id='cc624012-b274-4e90-93d2-1ed4faa271ee',
│   │   role='system',
│   │   content="Answer the user's question using the available tools. If you need to use a tool, call it with the appropriate arguments. If you have the answer, respond directly to the user.",
│   │   type='SystemMessage'
│   ),
│   UserMessage(
│   │   id='bae060cd-d883-4e93-949d-5bf0e270f575',
│   │   role='user',
│   │   content=['Can you search for Ravikumarchavva github profile and find top 3 repositories?'],
│   │   type='UserMessage',
│   │   name=None
│   ),
│   AssistantMessage(
│   │   id='6038ae04-8d68-41e5-8632-3d57e85095df',
│   │   role='assistant',
│   │   content=None,
│   │   type='AssistantMessage',
│   │   name=None,
│   │   reasoning=None,
│   │   tool_calls=[
│   │   │   ToolCallMessage(
│   │   │   │   id='call_tlRwdzk735vCL3mK9nKMGUvY',
│   │   │   │   role='tool_call',
│   │   │   │   content=None,
│   │   │   │   type='ToolCallMessage',
│   │   │   │   name='web_search_tool',
│   │   │   │   arguments={
│   │   │   │   │   'url': 'https://github.com/Ravikumarchavva',
│   │   │   │   │   'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
│   │   │   │   │   'max_chars': 1000,
│   │   │   │   │   'start_index': 0
│   │   │   │   }
│   │   │   )
│   │   ],
│   │   finish_reason='tool_calls',
│   │   usage=UsageStats(prompt_tokens=152, completion_tokens=87, total_tokens=239),
│   │   cached=False
│   ),
│   ToolExecutionResultMessage(
│   │   id='c0bbed89-0648-408c-9ab4-a3672f26327b',
│   │   role='tool_response',
│   │   content=[
│   │   │   {
│   │   │   │   'type': 'text',
│   │   │   │   'text': "[![View Ravikumarchavva's full-sized avatar](https://avatars.githubusercontent.com/u/109415623?v=4)](https://avatars.githubusercontent.com/u/109415623?v=4)\n\n## C.Ravi Kumar reddy Ravikumarchavva\n\n## Block or report Ravikumarchavva\n\n## Hi there! 👋\n\nI’m **Ravikumar Chavva**, a final-year Computer Science and Engineering student specializing in **Data Science** and **Machine Learning**. I’m passionate about leveraging data and AI to solve real-world problems and creating impactful solutions.\n\n---\n\n## 👩\u200d💻 About Me\n\n* 🎓 **Final-Year Student**: Pursuing B.Tech with a focus on Data Science and Machine Learning.\n* 🤖 **AI Enthusiast**: Passionate about exploring cutting-edge technologies and their practical applications.\n* 🌟 **Driven Problem-Solver**: Dedicated to building end-to-end solutions integrating ML, cloud, and full-stack technologies.\n\n---\n\n## 🛠️ Skills\n\n### Programming Languages\n\n* **Python** | **SQL** | **TypeScript**\n\n### Frameworks & Tools\n\n* **Machine Learning & AI:** Scikit-lear"
│   │   │   }
│   │   ],
│   │   type='ToolExecutionResultMessage',
│   │   tool_call_id='call_tlRwdzk735vCL3mK9nKMGUvY',
│   │   name='web_search_tool',
│   │   isError=False
│   ),
│   AssistantMessage(
│   │   id='93b6b74e-9e24-4f41-9c6f-80b43dba0a16',
│   │   role='assistant',
│   │   content=None,
│   │   type='AssistantMessage',
│   │   name=None,
│   │   reasoning=None,
│   │   tool_calls=[
│   │   │   ToolCallMessage(
│   │   │   │   id='call_ApvVUdDEYyL1f1HvJfFfdjji',
│   │   │   │   role='tool_call',
│   │   │   │   content=None,
│   │   │   │   type='ToolCallMessage',
│   │   │   │   name='web_search_tool',
│   │   │   │   arguments={
│   │   │   │   │   'url': 'https://github.com/Ravikumarchavva?tab=repositories',
│   │   │   │   │   'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
│   │   │   │   │   'max_chars': 1000,
│   │   │   │   │   'start_index': 0
│   │   │   │   }
│   │   │   )
│   │   ],
│   │   finish_reason='tool_calls',
│   │   usage=UsageStats(prompt_tokens=497, completion_tokens=91, total_tokens=588),
│   │   cached=False
│   ),
│   ToolExecutionResultMessage(
│   │   id='a8657c30-bac5-40e7-afda-ed55bd337274',